In [1]:
import pickle
import requests
from bs4 import BeautifulSoup

In [3]:
revs = "https://revistas.ulima.edu.pe/"
req_revs = requests.get(revs)
soup_revs = BeautifulSoup(req_revs.content, "lxml")

In [13]:
title = "Interfases"
journals = soup_revs.find("div", attrs="journals").find_all("li", attrs="has_thumb")
for journal in journals:
    titlet = journal.find("div", attrs="body").h3.a.string
    titlet = titlet.replace("\n", "").replace("\t", "").strip()
    if titlet == title:
        break
journal

<li class="has_thumb">
<div class="thumb">
<a href="https://revistas.ulima.edu.pe/index.php/Interfases">
<img src="https://revistas.ulima.edu.pe/public/journals/3/journalThumbnail_es_ES.jpg"/>
</a>
</div>
<div class="body">
<h3>
<a href="https://revistas.ulima.edu.pe/index.php/Interfases" rel="bookmark">
									Interfases
								</a>
</h3>
<div class="description">
<p><strong><acronym title="International Standard Serial Number" xml:lang="en">ISSN</acronym></strong>: 1993-4912 (Electrónico); <strong>DOI</strong>:  <a href="/index.php/interfases" target="_self">10.26439/interfases</a></p><br/>
<p style="text-align: justify;"><em>Interfases</em> es una revista digital, gestionado por la Universidad de Lima, para la publicación de investigaciones originales en áreas temáticas relacionadas con las ciencias de la computación, gestión de datos, aprendizaje automático, sistemas de información y áreas afines. Se publican artículos científicos y avances de investigación, siempre que cumplan 

In [17]:
link_journalt = journal.h3.a["href"]
req_journalt = requests.get(link_journalt)
soup_journalt = BeautifulSoup(req_journalt.content, "lxml")

In [24]:
link_journal = soup_journalt.find("div", attrs="current_issue").find("a", attrs="read_more")["href"]
req_journal = requests.get(link_journal)
soup_journal = BeautifulSoup(req_journal.content, "lxml")

In [64]:
soup_vols = soup_journal.find("ul", attrs="issues_archive row").find_all("div", attrs="obj_issue_summary")[1:]
vols = []
for soup_vol in soup_vols:
    dictt = {}
    link_vol = soup_vol.find("a", attrs="title")
    title = link_vol.string
    title = title.replace("\n", "").replace("\t", "").strip()
    temp = title.find("Núm.")+5
    temp2 = title.find("(")-1
    temp3 = title[temp:temp2].count("0")
    title = "Vol. "+title[:temp]+title[temp:temp2].replace("0","",temp3)+title[temp2:]
    vol_i = title.find("Vol.") + 4
    vol_d = title.find("Núm.")
    volumen = title[vol_i+1: vol_d-1]
    link_vol = link_vol["href"]
    archive = title.replace("ú", "u").replace(" ","_").replace("(", "").replace(")", "").replace(".", "")
    dictt["title"] = title
    dictt["link_vol"] = link_vol
    dictt["archive"] = archive
    vols.append(dictt)
vols

[{'title': 'Vol. Núm. 14 (2021)',
  'link_vol': 'https://revistas.ulima.edu.pe/index.php/Interfases/issue/view/351',
  'archive': 'Vol_Num_14_2021'},
 {'title': 'Vol. Núm. 13 (2020)',
  'link_vol': 'https://revistas.ulima.edu.pe/index.php/Interfases/issue/view/340',
  'archive': 'Vol_Num_13_2020'},
 {'title': 'Vol. Núm. 12 (2019)',
  'link_vol': 'https://revistas.ulima.edu.pe/index.php/Interfases/issue/view/323',
  'archive': 'Vol_Num_12_2019'},
 {'title': 'Vol. Núm. 11 (2018)',
  'link_vol': 'https://revistas.ulima.edu.pe/index.php/Interfases/issue/view/214',
  'archive': 'Vol_Num_11_2018'},
 {'title': 'Vol. Núm. 1 (2017)',
  'link_vol': 'https://revistas.ulima.edu.pe/index.php/Interfases/issue/view/143',
  'archive': 'Vol_Num_1_2017'},
 {'title': 'Vol. Núm. 9 (2016)',
  'link_vol': 'https://revistas.ulima.edu.pe/index.php/Interfases/issue/view/111',
  'archive': 'Vol_Num_9_2016'},
 {'title': 'Vol. Núm. 8 (2015)',
  'link_vol': 'https://revistas.ulima.edu.pe/index.php/Interfases/issue

In [90]:
for vol in vols:
    req_vol = requests.get(vol["link_vol"])
    soup_vol = BeautifulSoup(req_vol.content, "lxml")
    sections = soup_vol.find("div", attrs="sections").find_all("div", "section")
    if len(sections) > 1:
        sections = sections[1:]
    arts = []
    for section in sections:
        if "Artículos" in section.h2.string.replace("\n", "").replace("\t", ""):
            arts += section.find_all("div", attrs="obj_article_summary")
    artt = []
    for art in arts:
        dictt = {}
        link_art = art.find("div", attrs="title").a
        title = link_art.string
        title = title.replace("\n", "").replace("\t", "").strip()
        link_art = link_art["href"]
        authors = art.find("div", attrs="authors").string
        authors = authors.replace("\n", "").replace("\t", "").strip().split(",")
        dictt["authors"] = authors
        dictt["title"] = title
        dictt["link_art"] = link_art
        artt.append(dictt)
    vol["articles"] = artt

In [103]:
for vol in vols:
    for art in vol["articles"]:
        req_art = requests.get(art["link_art"])
        soup_art = BeautifulSoup(req_art.content, "lxml")
        date = soup_art.find("div", attrs="item published").find("div", attrs="value").string
        date = date.replace("\n", "").replace("\t", "").strip()
        art["date"] = date

In [104]:
with open("scrappULIMA.pickle", "wb") as f:
    pickle.dump(vols, f)

In [105]:
with open("scrappULIMA.pickle", "rb") as f:
    obj = pickle.load(f)
    print(obj)

[{'title': 'Vol. Núm. 14 (2021)', 'link_vol': 'https://revistas.ulima.edu.pe/index.php/Interfases/issue/view/351', 'archive': 'Vol_Num_14_2021', 'articles': [{'authors': ['Mauricio Alonso  Bejarano Reyes', ' Edgar  André Manzano Ramos'], 'title': 'Implementación de un sistema de monitoreo a nivel de prototipo de signos vitales: pulso, temperatura y saturación de oxígeno para pacientes', 'link_art': 'https://revistas.ulima.edu.pe/index.php/Interfases/article/view/5168', 'date': '2021-12-21'}, {'authors': ['Diana Valeria Paucar Bernardo', ' Paulina Tessalia Acho Santillan', ' Carlos Salvador Peralta Delgado'], 'title': 'Relación de la gestión de riesgos y calidad de software realizados por los profesionales del Colegio de Ingenieros del Perú del Consejo Departamental de Lima', 'link_art': 'https://revistas.ulima.edu.pe/index.php/Interfases/article/view/5111', 'date': '2021-12-21'}, {'authors': ['Marleny Peralta Ascue'], 'title': 'Calidad de datos en sistemas de gestión académica universi